In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping_nl-repository
sys.path.append("..")
import ribasim_lumping_nl

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
start_time = pd.Timestamp.now()

#### BASIS OPZETTEN

In [ ]:
# define network name, base dir
network_name = "HunzeenAas"

base_dir = Path("..\\..\\LHM\\")

# directory hydamo data
hydamo_basis_dir = Path(base_dir, "01_data_bewerking_analyse\\3_verwerkte_data", network_name)
# directory results
results_dir = Path(base_dir, "02_ribasim_netwerk_schematisatie\\", network_name)
# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [ ]:
# # define network name, base dir
# network_name = "tki_hooge_raam"
# base_dir = Path("..\\..\\ribasim_lumping_data\\")
# # directory D-HYDRO basis calculations
# dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")
# # directory D-HYDRO results calculations
# dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")
# # directory results
# results_dir = Path(base_dir, network_name, "3_ribasim\\")
# # batch file used to calculate volume D-Hydro module per GridPoint
# dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
# dhydro_volume_tool_increment = 0.1
# # link to ribasim_executable
# path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [ ]:
hydamo_basis_dir

In [ ]:
# Create networkanalysis
network = ribasim_lumping_nl.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    # dhydro_basis_dir=dhydro_basis_dir,
    # dhydro_results_dir=dhydro_results_dir,
    hydamo_basis_dir=hydamo_basis_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)
# network.export_to_geopackage()

#### INLEZEN HYDAMO  DATA

In [ ]:
hydamo_file = Path(hydamo_basis_dir,"hydamo.gpkg")
# gpd.read_file(hydamo_file, layer='peilgebiedpraktijk')

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='hydamo',
    hydamo_basis_dir = hydamo_basis_dir
);

Define locations where the network should be split into Ribasim basins:

In [ ]:
# read files with split locations
split_file = Path(results_dir,"Duurswold.gdb")
splitlocs = gpd.GeoDataFrame()
for layer in fiona.listlayers(split_file):
    print(layer)
    data = gpd.read_file(split_file,layer=layer)
    data = data[['geometry']]
    data_point = data.loc[data.geometry.geometry.type=='Point'] # only keep points (TODO work with culverts which are linestrings)
    splitlocs = pd.concat([splitlocs, data_point])

# merge list with split locations with structures
splitlocs = gpd.sjoin_nearest(splitlocs,pd.concat([network.weirs_gdf, network.pumps_gdf]), how= 'left', max_distance=1 ) # TODO also merge with culverts
splitlocs = splitlocs.loc[splitlocs['structure_id'].notna()]


In [ ]:
splitlocs

In [ ]:
network.add_split_nodes(
    # weirs=True,
    # pumps=False,
    # uniweirs=False,
    # edges=False,
    structures_ids_to_include=splitlocs.structure_id,
    # structures_ids_to_exclude=[],
    # edge_ids_to_include=[],                                  
    # edge_ids_to_exclude=[],
);

In [ ]:
# temp action. add edge number to splitnodes. TODO integrate in script
columns_splitnod = network.split_nodes.columns
temp = gpd.sjoin(network.split_nodes, network.edges_gdf[['geometry','edge_no']], how= 'left')
temp['edge_no'] = temp['edge_no_right']
temp = temp.groupby('split_node').first().reset_index()
temp = temp.loc[temp['edge_no'].notna()] # structures die niet goed op de edge liggen: TODO snapping
temp = temp.drop_duplicates(subset=['edge_no']) # meerdere structures op dezelfde edge
temp = temp.set_crs(28992)
temp = temp[columns_splitnod]


In [ ]:
network.split_nodes = temp

Specify translation split_nodes to ribasim-nodes per type or for specifiek split_node_id

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    uniweir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)
split_node_id_conversion = {}

Create basins (gdf) based on nodes, edges, split_node_ids and areas

In [ ]:
simulation_code = 'test_duurswold'

In [ ]:
network.nodes_gdf = network.nodes_gdf.set_crs(28992) 

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);